In [3]:
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import pickle

In [4]:
# open text file
with open('dataset/train.ft.txt', 'r', encoding='utf8') as f:
    txt = [a for a in f]

In [5]:
df = pd.DataFrame()
df['text'] = txt
df['label'], df['text'] = df.text.apply(lambda x: x.split(' ', maxsplit=1)[0]), df.text.apply(lambda x: x.split(' ', maxsplit=1)[1])
df.label = df.label.map({'__label__2':1, '__label__1':0})

In [6]:
df_train = df[0:3000000]
df_valid = df[3000000:3300000]
df_test = df[3300000:3600000]

In [7]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [9]:
GET_NEW_TOKENIZER = True
if GET_NEW_TOKENIZER:
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=512, char_level=True)
    tokenizer.fit_on_texts(tqdm(df_train.text))
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

100%|██████████| 3000000/3000000 [02:21<00:00, 21185.66it/s]


In [10]:
train_features = tokenizer.texts_to_matrix(tqdm(df_train.text))

 33%|███▎      | 975519/3000000 [00:44<01:22, 24531.25it/s]

In [ ]:
train_target = df_train.label